Вытащила стол6ец с ID инвертонов из та6лицы Supplement, сохранила как invertonsID.txt Написала скрипт для вычленения уникальных ID:

In [ ]:
# Создаем множество для хранения уникальных основных ID
unique_ids = set()

# Чтение ID из файла
with open("invertonID.txt", "r") as file:
    for line in file:
        # Удаляем пробелы и символы новой строки
        id_ranges = line.strip()
        # Разделяем строку по символу ':'
        genome_id, _ = id_ranges.split(':')
        # Добавляем основной ID в множество
        unique_ids.add(genome_id)

# Преобразуем множество в список, если нужно
unique_id_list = list(unique_ids)

# Сохраняем уникальные основные ID в новый текстовый файл
with open("invertonID_unique.txt", "w") as output_file:
    for unique_id in unique_id_list:
        output_file.write(unique_id + "\n")


Список полученных ID использовала для скачивания при помощи entrez, цикл написала в скрипте в питоне. 

In [ ]:
#!/usr/bin/env python3
import time
from Bio import Entrez

# Адрес электронной почты
Entrez.email = "iduvzavtra@gmail.com"

# Чтение ID и участков из файла
with open("invertonID_unique.txt", "r") as file:
    for line in file:
        genome_id = line.strip()

        # Загрузка последовательности
        handle = Entrez.efetch(db="nucleotide", id=genome_id, rettype="fasta", retmode="text")
        genome_sequence = handle.read()
        handle.close()

        # Сохранение последовательность в файл
        with open(f"{genome_id}.fasta", "w") as output_handle:
            output_handle.write(genome_sequence)

        # Задержка после обработки каждого ID
        time.sleep(5)

**Косяк:** не указала папку для сохранения, валятся на ра6очий стол, руками перетаскиваю в entrez_req. Сильно недооценила время выполнения. Запуск из командной строки Windows.

In [ ]:
#!/usr/bin/env python3
import os
import time
from Bio import Entrez

# Адрес электронной почты
Entrez.email = "iduvzavtra@gmail.com"

# Папка для сохранения загруженных файлов
output_folder = "entrez_req"

# Чтение ID из файла
with open("invertonID_unique.txt", "r") as file:
    for line in file:
        genome_id = line.strip()
        fasta_file_path = os.path.join(output_folder, f"{genome_id}.fasta")

        # Проверка, существует ли файл
        if os.path.exists(fasta_file_path):
            print(f"Файл {fasta_file_path} уже существует. Пропускаем загрузку для ID {genome_id}.")
            continue  # Пропускаем загрузку, если файл уже существует

        # Загрузка последовательности
        handle = Entrez.efetch(db="nucleotide", id=genome_id, rettype="fasta", retmode="text")
        genome_sequence = handle.read()
        handle.close()

        # Сохранение последовательности в файл
        with open(fasta_file_path, "w") as output_handle:
            output_handle.write(genome_sequence)

        # Задержка после обработки каждого ID
        time.sleep(5)

Исправленная версия - проверяется, есть ли файл в папке, и скачивание идет сразу в папку.

Написала код для вырезания фрагментов. Настроено исключение для файлов, которые еще не скачались, хорошо 6ы такие ID выписывать в отдельный txt, но увы, это не 6ыло продумано.

In [ ]:
import os
from Bio import SeqIO

input_folder = "entrez_req"  # Папка, где находятся исходные FASTA-файлы
output_folder = "invertons_cut"  # Папка, куда будут сохраняться вырезанные фрагменты

with open("invertonID.txt", "r") as file:
    for line in file:
        id_ranges = line.strip()  
        genome_id, ranges = id_ranges.split(':')
        positions = ranges.split('-')

        fasta_file_path = os.path.join(input_folder, f"{genome_id}.fasta")
        
        # Попробуем открыть файл, если он не существует, пропустим этот ID
        try:
            with open(fasta_file_path, "r") as fasta_file:
                fasta_record = SeqIO.read(fasta_file, "fasta")
                full_sequence = fasta_record.seq

            # Получаем и сохраняем участки с учетом смещения
            for i in range(0, len(positions), 2):  
                start = int(positions[i]) - 20000  
                end = int(positions[i + 1]) + 20000 

                # Извлекаем последовательность
                fragment = full_sequence[start:end]

                # Сохраняем последовательности в отдельные файлы в выходной папке
                fragment_file_path = os.path.join(output_folder, f"{genome_id}_{start}_{end}.fasta")
                with open(fragment_file_path, "w") as output_handle:
                    output_handle.write(f">fragment_{genome_id}_{start}_{end}\n")
                    output_handle.write(str(fragment) + "\n")
        
        except FileNotFoundError:
            print(f"Файл {fasta_file_path} не найден. Пропускаем ID {genome_id}.")

**Версия 2.0:** код упал, потому что я не удалила файл с 6олее раннней попытки с длинным именем, вписала проверку на наличие фрагмента и вывод ID в отдельный файл. Теперь из одного ID один файл - только по минимальной имаксимальной координате.

In [ ]:
import os
from Bio import SeqIO

input_folder = "entrez_req"  # Папка, где находятся исходные FASTA-файлы
output_folder = "invertons_cut"  # Папка, куда будут сохраняться вырезанные фрагменты
missing_ids_file = "missing_ids.txt"  # Файл для записи ID, для которых не найдены FASTA

# Получаем список существующих файлов фрагментов
existing_fragments = set(os.listdir(output_folder))

# Открываем файл для записи ID, для которых не найдены FASTA
with open(missing_ids_file, "w") as missing_ids:
    # Читаем ID из файла "invertonID.txt"
    with open("invertonID.txt", "r") as file:
        for line in file:
            id_ranges = line.strip()
            genome_id, ranges = id_ranges.split(':')
            positions = list(map(int, ranges.split('-')))  # Преобразуем строки в целые числа

            fasta_file_path = os.path.join(input_folder, f"{genome_id}.fasta")

            # Проверяем, существует ли FASTA файл
            try:
                with open(fasta_file_path, "r") as fasta_file:
                    fasta_record = SeqIO.read(fasta_file, "fasta")
                    full_sequence = fasta_record.seq
            except FileNotFoundError:
                missing_ids.write(genome_id + "\n")
                continue  # Пропускаем, если файл не найден

            # Находим минимальное и максимальное значение координат
            start = min(positions) - 20000
            end = max(positions) + 20000
            
            fragment_file_name = f"{genome_id}_{start}_{end}.fasta"
            
            # Если фрагмент уже существует, пропускаем
            if fragment_file_name in existing_fragments:
                continue
            
            # Извлекаем последовательность
            fragment = full_sequence[start:end]

            # Сохраняем последовательность в отдельные файлы в выходной папке
            fragment_file_path = os.path.join(output_folder, fragment_file_name)
            with open(fragment_file_path, "w") as output_handle:
                output_handle.write(f">fragment_{genome_id}_{start}_{end}\n")
                output_handle.write(str(fragment) + "\n")